In [1]:
%pip install request

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
Note: you may need to restart the kernel to use updated packages.


## NDPH

In [ ]:
import requests
import time

In [ ]:
# Earliest date (>1756)
# Latest date (<1968)

def NDPH_request(query, earliest, latest):
    # url = f"https://chroniclingamerica.loc.gov/search/pages/results/?date1={earliest}&date2={latest}&title={query}&format=json" #search titles NOT WORKING
    url = f"https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1={earliest}&date2={latest}&proxtext={query}&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic&format=json"
    print(f"dates: {earliest} to {latest} and query: {query}")
    response = requests.get(url)
    print(url)
    return response.json()

In [23]:
def find_state(state):
    process_name = state.replace(" ", "+")
    print(f"Finding state: {process_name}")
    response = requests.get(f"https://chroniclingamerica.loc.gov/search/titles/results/?terms={process_name}&format=json")
    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")
    
    # Parse the JSON response
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        raise ValueError(f"Failed to decode JSON from response: {response.text}")
    return response.json()

In [ ]:
def check_year(state,  latest, union, page):
    for item in state['items']:
        if item['end_year'] < latest:
            union.append(item)
    if state['endIndex'] == state['totalItems']:
        return union
    print(f"current page: {page} and current index: {state['endIndex']}")
    return check_year(state, latest, union, page + 1)

In [ ]:
def find_newspapers(state_name, year, union, page):
    print(f"the state name is {state_name} and the year is {year}")
    info = requests.get(f"https://chroniclingamerica.loc.gov/search/titles/results/?terms={state_name}&page={page}&format=json")
    if info.status_code != 200:
        raise ValueError(f"Request failed with status code {info.status_code}: {info.text}")
    
    state_info = info.json()
    
    for item in state_info['items']:
        if item['start_year'] < year:
            union.append(item)
    if state_info['endIndex'] == state_info['totalItems']:
        return union
    print(f"current page: {page} and current index: {state_info['endIndex']}")

    time.sleep(.25)
    
    return find_newspapers(state_name, year, union, page + 1)

In [30]:
def find_states(selected_states):
    american_newspapers = {}
    for state, year in selected_states.items():
        newspaper_list = []
        # print(f"Finding newspapers for {state} in {year}")
        american_newspapers[state] = find_newspapers(state, year - 1, newspaper_list, 1)
    return american_newspapers


In [58]:
states_founded = {
    "Delaware": 1787,
    "Pennsylvania": 1787,
    "New Jersey": 1787,
    "Georgia": 1788,
    "Connecticut": 1788,
    "Massachusetts": 1788,
    "Maryland": 1788,
    "South Carolina": 1788,
    "New Hampshire": 1788,
    "Virginia": 1788,
    "New York": 1788,
    "North Carolina": 1789,
    "Rhode Island": 1790,
    "Vermont": 1791,
    "Kentucky": 1792,
    "Tennessee": 1796,
    "Ohio": 1803,
    "Louisiana": 1812,
    "Indiana": 1816,
    "Mississippi": 1817,
    "Illinois": 1818,
    "Alabama": 1819,
    "Maine": 1820,
    "Missouri": 1821,
    "Arkansas": 1836,
    "Michigan": 1837,
    "Florida": 1845,
    "Texas": 1845,
    "Iowa": 1846,
    "Wisconsin": 1848,
    "California": 1850,
    "Minnesota": 1858,
    "Oregon": 1859,
    "Kansas": 1861,
    "West Virginia": 1863,
    "Nevada": 1864,
    "Nebraska": 1867,
    "Colorado": 1876,
    "North Dakota": 1889,
    "South Dakota": 1889,
    "Montana": 1889,
    "Washington": 1889,
    "Idaho": 1890,
    "Wyoming": 1890,
    "Utah": 1896,
    "Oklahoma": 1907,
    "New Mexico": 1912,
    "Arizona": 1912,
    "Alaska": 1959,
    "Hawaii": 1959
}

In [49]:
test_states = {
    "Alaska": 1959
}
result = find_states(test_states)
print(result)

the state name is Alaska and the year is 1958
here
Rate limit hit. Waiting for 1 seconds...


ValueError: Failed after multiple retries.

In [50]:
test_state = "Alaska"
test_info = find_state(test_state)
test_info
# state_lists = []
# state_lists = find_newspapers(test_state, 1889, state_lists, 1)
# check_year(test_info, 1958, state_lists, 1)
# state_lists


Finding state: Alaska


ValueError: Request failed with status code 429: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-ima

In [41]:
# test_info
# len(state_lists)
state_lists

[{'essay': ['\n<p>\r\n\tThe <em>Alaska Juneau-Douglas Island news</em> was a bimonthly publication published on the first and fifteenth of each month by the Alaska Treadwell and Alaska Juneau Gold Mining Companies. The publication ran from September 1, 1918 to January 15, 1919, and the editor was L.S. Ferris, also called Steve Ferris, who was in charge of the employment bureau for the Treadwell Company. The publication focused on workers who were serving with the armed forces during World War I and ran the line "for the boys \'over there\'" in the masthead. The paper explained in its first issue that the publication was "intended to be a community letter... to all former employees of the Alaska Juneau, the Alaska Treadwell, the Alaska Mexican and the Alaska United Gold Mining Companies who are bearing arms in our country\'s service."</p>\n<p>\r\n\tThe newspaper provided updates on the soldiers, kept an honor roll of workers who were serving, and maintained a service flag with stars for

In [10]:
for articles in state_lists:
    print(articles['title'])
    print(articles['start_year'])
    print(articles['end_year'])

The Alaska Juneau-Douglas Island news. [volume]
1918
1919
The Alaska fisherman. [volume]
1923
1932
Alaska resident. [volume]
1930
1999
Alaska daily capital. [volume]
1920
1920
Alaska labor dispatch. [volume]
1935
1999
[Alaska wireless bulletin]. [volume]
1915
1999
Daily prospector bulletin.
1905
1911
The daily Alaska citizen.
1916
1920
The Alaska advocate. [volume]
1943
1949
Alaska daily guide. [volume]
1902
1905
The Alaska truth. [volume]
1899
1999
The Alaska times.
1913
1999
The Alaska Sunday morning post. [volume]
1912
1915
The Alaska journal. [volume]
1893
1893
Polar star.
1946
1989
S.S. Alaska radio news.
1927
1927
Alaska miners news.
1923
1923
Alaska labor union bulletin.
1918
1999
The Alaska citizen.
1910
1917
The weekly Alaska citizen.
1917
1920
The Alaska sourdough. [volume]
1911
1911
The Daily Alaska empire. [volume]
1926
1964
The Alaska daily empire. [volume]
1912
1926
Anchorage weekly times and Alaska labor news. [volume]
1917
1918
The Alaska socialist.
1913
1999
The Alaska

In [11]:
for item in test1_data["items"]:
    if item["end_year"] <= 1900:
        print(item["title"], item["start_year"])

The Texas monument. [volume] 1850
The Texas Republican. [online resource] 1849
The Texas tribune. [volume] 1887
Texas almanac "extra." [volume] 1862


Can use the json arg to get the 
To get the actual information inside of the api call, we have to actually make multiple api calls. But we also have to use rest to not get a denial of service
REST APIs? 
For this action 

In [ ]:
test = requests.get("https://chroniclingamerica.loc.gov/search/titles/results/?terms=texas&page=22&format=json")
test.json()
testdata = test.json()
testdata

{'totalItems': 1093,
 'endIndex': 1093,
 'startIndex': 1051,
 'itemsPerPage': 20,
 'items': [{'essay': [],
   'place_of_publication': 'Houston [Tex.]',
   'start_year': 1863,
   'publisher': '[publisher not identified]',
   'county': ['Harris'],
   'edition': None,
   'frequency': 'Three times a week',
   'url': 'https://chroniclingamerica.loc.gov/lccn/sn84022298.json',
   'id': '/lccn/sn84022298/',
   'subject': ['Galveston (Tex.)--Newspapers.',
    'Houston (Tex.)--Newspapers.',
    'Texas--Galveston.--fast--(OCoLC)fst01212106',
    'Texas--Houston.--fast--(OCoLC)fst01205077'],
   'city': ['Houston'],
   'language': ['English'],
   'title': 'News bulletin. [volume]',
   'holding_type': ['Microfilm Service Copy',
    'Original',
    'Microfilm Service Copy',
    'Original',
    'Original',
    'Original',
    'Original',
    'Microfilm Service Copy',
    'Microfilm Master'],
   'end_year': 1865,
   'alt_title': [],
   'note': ['Also issued on microfilm from The University of Texas, Ce

In [ ]:
# Psuedocode from meeting w Charles
def make_api_calls(state_name, page=1, years=[]):
   result = request.get(f”{url}?term={state_name}&page={page}”)
   for item in result[“items”]: # going through the items attr/list
      years.append(item[“date”][0:5]) # get the year/date attribute
   if result[“endIndex”] == result[“totalItems”]: # end index is where the page ends
      return years # so if we're at tje end of the page, return the years and end the function
   return make_api_calls(state_name, page+1, years) # recursive call where we add 1 to the page number

In [ ]:
def get_years_1state(state_name, year_founded):
   result = NDPH_request(state_name, 1756, year_founded -1)
   for item in result["items"]